#Fake News Prediction using Machine Learning with Python and inference system



1: Fake news
0: real news

In [61]:
#Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer



In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

data pre-processing

In [64]:
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningCourseWithPython/data/train.csv')

In [65]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [66]:
news_dataset.shape

(20800, 5)

In [67]:
news_dataset.isnull().sum() #number of missing values

,0
id,0
title,558
author,1957
text,39
label,0


In [68]:
news_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [69]:
news_dataset = news_dataset.fillna('')  #replacing the null values with empty strings

In [70]:
# merging the author name and news title

news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [71]:
news_dataset['content'].head()


,content
0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Consortiumnews.com Why the Truth Might Get You...
3,Jessica Purkiss 15 Civilians Killed In Single ...
4,Howard Portnoy Iranian woman jailed for fictio...


In [72]:
X = news_dataset['content']
Y = news_dataset['label']


In [73]:
print(X)
print(Y)

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming

a process of reducing a word to its root word

example:
actor,actress,acting --> act

In [74]:
porter_stem = PorterStemmer()
#

In [75]:
stop_words = set(stopwords.words('english'))

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower().split()
    stemmed_content = [
        porter_stem.stem(word)
        for word in stemmed_content
        if word not in stop_words
    ]
    return ' '.join(stemmed_content)


In [77]:
news_dataset['content'] = news_dataset['content'].apply(stemming)


In [78]:
print(news_dataset['content'].head())

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object


In [79]:
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [80]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [81]:
Y.shape


(20800,)

In [83]:
# print(X)

In [84]:
#spliting training and testing data
#X :features Y:labels

X_train, X_test, Y_train, Y_test = train_test_split(
    news_dataset['content'],
    news_dataset['label'],
    test_size=0.2,
    stratify=news_dataset['label'],
    random_state=2
)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [85]:
print(X.shape,X_train.shape,X_test.shape)

(20800,) (16640, 15495) (4160, 15495)


In [86]:
#Training the model

model = LogisticRegression(
    max_iter=1000,
    solver='liblinear'
)


In [87]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [88]:

#Evaluation
#accuracy score on traininig data

X_train_prediction=model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

print('Accuracy score of the training data : ', training_data_accuracy)


Accuracy score of the training data :  0.9864783653846154


In [89]:
#Evaluation
#accuracy score on testing data

X_test_prediction=model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

print('Accuracy score of the testing data : ', testing_data_accuracy)

Accuracy score of the testing data :  0.9800480769230769


In [90]:
#inference system
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [92]:
# print(Y_test[1])

In [93]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test, X_test_prediction))
print(confusion_matrix(Y_test, X_test_prediction))


              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2077
           1       0.97      0.99      0.98      2083

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160

[[2011   66]
 [  17 2066]]


infernecne system

In [94]:
def predict_news(news_text):
    # preprocess
    processed_text = stemming(news_text)

    # vectorize
    vectorized_text = vectorizer.transform([processed_text])

    # prediction
    prediction = model.predict(vectorized_text)[0]

    # output
    if prediction == 1:
        return "Fake News"
    else:
        return "Real News"


In [99]:
news_example = "The New York city council passed a new law affecting public transportation on Monday."
result = predict_news(news_example)
print("Prediction:", result)


Prediction: Real News


In [102]:
processed = stemming("The New York city council passed a new law affecting public transportation on Monday.")
vectorized = vectorizer.transform([processed])

print(model.predict_proba(vectorized))


[[0.96068835 0.03931165]]


In [100]:
news_example = "Donkeys were flying in space"
result = predict_news(news_example)
print("Prediction:", result)


Prediction: Fake News


In [101]:
processed = stemming("Donkeys were flying in space")
vectorized = vectorizer.transform([processed])

print(model.predict_proba(vectorized))


[[0.12284371 0.87715629]]
